# Calculo Porcentajes de uso del suelo

Este script abarca el flujo de trabajo completo para obtener una tabla con el porcentaje de ocupación de uso del suelo en las ciudades de más de 100.000 habitantes de España, excluyendo el uso sin edificar y otros. Los pasos de este flujo de trabajo son los siguientes: 

1.- crear nueva columna para la clase agregada
2.- reclasificar los datos a una clase agregada
3.- asignar un nuevo codigo numérico a las clases agregadas 
4.- exportar estos shapefile con las modificaciones realizadas hasta ahora
5.- eliminar las clases sin edificar y otros.
6.- calcular los porcentajes de ocupación de cada uso respecto al total, sin considerar sin edificar y otros. 

In [30]:
import pandas as pd
import geopandas as gpd
import os
import glob
import string

Vamos a configurar la representación de los números decimales 

### Este es un paquete que sirve para poder visualiar una barra de progreso

In [31]:
# Este es un paquete que sirve para poder hacer la barra de progreso

from tqdm import tqdm, trange 
from time import sleep
from tqdm.notebook import tqdm

### Definir directorio de trabajo donde se encuentran los shapes reclasificados utilizando la libreria os 

In [32]:
path =(r"F:/Respaldo toshiba/projectos/shapes/marcos_clasificacion_ag/")
# Change the directory
os.chdir(path)

Leer los shapefiles en un diccionario de forma iterativa.

In [33]:
# Creamos un diccionario vacío
dict_shapes = {}
# Barra de progroso con el numero total de elementos
pbar = tqdm(total = 72) 
# Iterate over all the files in the directory
for file in os.listdir():
   if file.endswith('.shp') and file[0].isalpha():
       temp = gpd.read_file(file)
       dict_shapes[file] = temp
       pbar.update(1)
       sleep(0.0001)
pbar.close()

  0%|          | 0/72 [00:00<?, ?it/s]

Creamos un nuevo diccionario, dict_areas, con los shapes incluyendo las clases agregadas para cada geometria, que no viene incluida dentro de los atributos del shape file original

In [35]:
dict_areas = {}
for key, value in dict_shapes.items():
    temps = dict_shapes[key].copy()
    temps['Area'] = temps.area
    dict_areas[key] = temps

1. Agregar clases usando códigos comunes entre categorias

Leer archivos csv con los códigos agregados

In [36]:
clases_agregadas = pd.read_csv(r"F:/Respaldo toshiba/projectos/shapes/clases_agregadas.csv")
clases_agregadas

,USO,USO_AG
0,COM,COM
1,ED_SING,ED_SING
2,EQUIP_EDU,EQUIP
3,EQUIP_OTR,EQUIP
4,EQUIP_SANI,EQUIP
5,HOS_REST,OCIO
6,IND,IND
7,IND_MX,IND
8,OCIO_ESP,OCIO
9,OFI,OFI


Definimos una función para hacer una join usando la función merge de pandas. Hacemos un leftjoin, a partir de la columna común uso.

In [37]:
def uso_agregado(data_in, clases):
    data_out = {}
    for key, value in data_in.items():
        data_out[key] = value.merge(clases, on = 'USO', how = 'left')
    return data_out


Aplicamos la función y obtenemos un nuevo diccionario con el uso agregado

In [38]:
uso_agregado = uso_agregado(dict_areas,clases_agregadas)

In [51]:
uso_agregado_albacete

{'Albacete_class.shp':        index  AREA          REFCAT  A_TOT_EDIF  SUP_ED_0  SUP_ED_1  \
 0          1  2138  6879902WJ9167H        5918       939      1177   
 1          3   647  6409719WJ9260N         809       648         0   
 2          4   250  3984813WJ9038S         275        95       100   
 3          5   375  7654308WJ9175D         196       196         0   
 4          6   649  3686015WJ9038N         342       286        56   
 ...      ...   ...             ...         ...       ...       ...   
 13779  16331  1496  6883027WJ9168S         391       391         0   
 13780  16332  4733  6883026WJ9168S        4627      4627         0   
 13781  16333   445  0095504WH8999S         154       154         0   
 13782  16336  1445  2239008WJ9223N         257       257         0   
 13783  16337   198  8478011WJ9187G         887       199       172   
 
        MAX_PLANTA  NUM_INM_R         USO  PROP_AE_AT  ...  HOS_REST  ED_SING  \
 0               4         29     RES_PLU  

Definimos una función para asignar un codigo por categoria agregada

uso_agregado

In [54]:
def asignacion_codigo_categorias(data_in):
    data_out = {}
    for key, df in data_in.items():
        df = df.reset_index()
        usos = df['USO_AG'].unique()
        cod_usos = np.arange(len(usos))
        cod_map = pd.Series(cod_usos, index=usos)
        df['cod_usos_a'] = df['USO_AG'].map(cod_map)
        data_out[key] = df
    return data_out

Usamos el diccionario uso agregado

In [55]:
uso_agregado_cat = asignacion_codigo_categorias(uso_agregado)

KeyError: 'USO_AG'

In [ ]:
Calculamos la suma de las areas de cada categoria agregada

In [26]:
df_final = pd.DataFrame()
for key, value in uso_agregado_cat.items():
    temp = uso_agregado_cat[key] 
    temp_cat = temp.groupby('USO_AG')['Area'].sum()
    temp_area = temp.groupby('USO_AG')['Area'].sum().sum()
    temp_porcentaje = (temp_cat/temp_area)*100
    df_temp = temp_porcentaje.to_frame().rename(columns = {'Area':key})
    df_final = pd.concat([df_final,df_temp], axis = 1)

NameError: name 'uso_agregado_cat' is not defined

# Exportar los datos a un csv

Definimos una función para eliminar la categoría otros de todos los shapes.

In [ ]:
dict_filter_sin_otros = {}
for key, value in uso_agregado_gtp_cat.items():
    ciudad = uso_agregado_gtp_cat[key].copy()
    ciudad1 = ciudad[ciudad.loc[:,'USO_AG'] != "OTROS"].copy()
    dict_filter_sin_otros[key] = ciudad1

 Volvemos a calcular los porcentajes sin la categoría otros 

In [29]:
df_sin_otros = pd.DataFrame()
for key, value in dict_filter_otros.items():
    temp = dict_filter_otros[key] 
    temp_cat = temp.groupby('USO_AG')['Area'].sum()
    temp_area = temp.groupby('USO_AG')['Area'].sum().sum()
    temp_porcentaje = (temp_cat/temp_area)*100
    df_temp = temp_porcentaje.to_frame().rename(columns = {'Area':key})
    df_otros = pd.concat([df_sin_otros,df_temp], axis = 1)

NameError: name 'dict_filter_otros' is not defined

Finalmente, volvemos a calcular los porcentajes de cada categoría respecto al total pero sin considerar la categoría otros, y sin edificación

In [ ]:
dict_filter = {}
for key, value in uso_agregado_gtp_cat.items():
    ciudad = uso_agregado_gtp_cat[key].copy()
    ciudad1 = ciudad[ciudad.loc[:,'USO_AG'] != "SIN_EDIF"].copy()
    ciudad2 = ciudad1[ciudad1.loc[:,'USO_AG'] != "OTROS"].copy()
    dict_filter[key] = ciudad2

In [ ]:
df_final_filter = pd.DataFrame()
for key, value in dict_filter.items():
    temp = dict_filter[key] 
    temp_cat = temp.groupby('USO_AG')['Area'].sum()
    temp_area = temp.groupby('USO_AG')['Area'].sum().sum()
    temp_porcentaje = (temp_cat/temp_area)*100
    df_temp = temp_porcentaje.to_frame().rename(columns = {'Area':key})
    df_final_filter = pd.concat([df_final_filter,df_temp], axis = 1)

In [ ]:
df_final_filter.to_csv('')

# FIN